In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[4]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.00,0.088523,-2.372,0.0,0.0
1,5957.57,0.587007,-2.126,0.0,0.0
2,5878.87,0.580422,-1.839,0.0,0.0
3,5827.80,0.576149,-1.521,0.0,0.0
4,5730.89,0.568040,-1.223,0.0,0.0
...,...,...,...,...,...
43819,6255.85,0.611964,0.048,0.0,0.0
43820,6191.86,0.606610,-0.163,0.0,0.0
43821,6070.38,0.596446,-0.274,0.0,0.0
43822,5822.73,0.575724,-0.311,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[4]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_CZ,Temp_CZ
0,0.00,-2.372
1,5957.57,-2.126
2,5878.87,-1.839
3,5827.80,-1.521
4,5730.89,-1.223
...,...,...
43819,6255.85,0.048
43820,6191.86,-0.163
43821,6070.38,-0.274
43822,5822.73,-0.311


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_CZ,Temp_CZ,Lag-2,Lag-7
0,7549.63,-4.644,7192.18,0.00
1,7485.12,-4.709,7120.78,5957.57
2,7378.78,-4.749,6992.29,5878.87
3,7443.10,-4.821,7062.87,5827.80
4,7879.07,-4.843,7480.10,5730.89
...,...,...,...,...
43651,6255.85,0.048,7185.40,5715.00
43652,6191.86,-0.163,6888.20,5604.65
43653,6070.38,-0.274,6711.40,5654.16
43654,5822.73,-0.311,6416.28,5437.48


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0123 - mape: 16809.3848 - mae: 0.0827 - val_loss: 0.0054 - val_mape: 8.5788 - val_mae: 0.0615
Epoch 2/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0087 - mape: 15719.1895 - mae: 0.0733 - val_loss: 0.0053 - val_mape: 9.1590 - val_mae: 0.0607
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0079 - mape: 15858.2383 - mae: 0.0691 - val_loss: 0.0065 - val_mape: 9.1019 - val_mae: 0.0669
Epoch 4/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0059 - mape: 15667.5781 - mae: 0.0563 - val_loss: 0.0126 - val_mape: 13.7898 - val_mae: 0.1005
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0048 - mape: 16247.4531 - mae: 0.0484 - val_loss: 0.0147 - val_mape: 14.9343 - val_mae: 0.1088
Epoch 6/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0043 - mape: 14959.9160 - mae: 0.0450 - val_loss: 0.0

1105/1105 [==============================] - 8s 8ms/step - loss: 0.0020 - mape: 3188.6094 - mae: 0.0276 - val_loss: 0.0085 - val_mape: 10.9923 - val_mae: 0.0798
Epoch 49/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0021 - mape: 4575.0142 - mae: 0.0280 - val_loss: 0.0087 - val_mape: 11.2215 - val_mae: 0.0816
Epoch 50/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0020 - mape: 4417.4956 - mae: 0.0271 - val_loss: 0.0069 - val_mape: 9.5894 - val_mae: 0.0698
Epoch 51/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0019 - mape: 3035.0552 - mae: 0.0271 - val_loss: 0.0089 - val_mape: 10.9674 - val_mae: 0.0806
Epoch 52/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0020 - mape: 2943.1782 - mae: 0.0271 - val_loss: 0.0103 - val_mape: 11.7184 - val_mae: 0.0861
Epoch 53/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0021 - mape: 4342.7192 - mae: 0.0279 - val_loss: 0.0078 - val_ma

1105/1105 [==============================] - 8s 8ms/step - loss: 0.0015 - mape: 2730.5974 - mae: 0.0240 - val_loss: 0.0152 - val_mape: 14.8003 - val_mae: 0.1063
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0017 - mape: 3836.5977 - mae: 0.0247 - val_loss: 0.0111 - val_mape: 12.5840 - val_mae: 0.0908
Epoch 97/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0015 - mape: 3456.1821 - mae: 0.0238 - val_loss: 0.0152 - val_mape: 14.8123 - val_mae: 0.1077
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0014 - mape: 2930.0188 - mae: 0.0232 - val_loss: 0.0129 - val_mape: 13.3786 - val_mae: 0.0973
Epoch 99/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0015 - mape: 2641.3721 - mae: 0.0234 - val_loss: 0.0113 - val_mape: 12.3588 - val_mae: 0.0892
Epoch 100/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0017 - mape: 2893.4419 - mae: 0.0249 - val_loss: 0.0135 - val_

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 856.0361888408661


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.5291238 , 0.5145987 , 0.4979536 , ..., 0.6001205 , 0.5869183 ,
       0.57691616], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.58507135, 0.58092873, 0.5809714 , ..., 0.59644577, 0.57572431,
       0.55806777])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,5934.44,5265.789551
1,5884.93,5092.194824
2,5885.44,4893.262695
3,6091.43,4778.916992
4,6858.88,4745.947754
...,...,...
4363,6255.85,6455.327637
4364,6191.86,6283.231934
4365,6070.38,6114.297852
4366,5822.73,5956.513672


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

12.27946735546909